In [36]:
# fox and dhole with mm2 asm20

all_dog_retrocopies = []
f = open("final_rcs_in_origin_coords.bed", 'rt')
inFile = f.readlines()
for line in inFile:
    line = line.rstrip().split()
    all_dog_retrocopies.append((line[0], line[1], line[2], line[3]))
f.close()
all_dog_retrocopies

In [37]:
f = open("mischka.blat_retrocopies_assembled.sorted.txt", 'rt')
mischka_rc_flanks = {}
inFile = f.readlines()
for line in inFile:
    line = line.rstrip().split()
    mischka_rc_flanks[(line[0], line[1], line[2])] = [(line[0], str(int(line[1])-1000), line[1]),(line[0], line[2], str(int(line[2])+1000))]
f.close()
mischka_rc_flanks

{('chr1', '325443', '330783'): [('chr1', '324443', '325443'),
  ('chr1', '330783', '331783')],
 ('chr1', '408403', '409542'): [('chr1', '407403', '408403'),
  ('chr1', '409542', '410542')],
 ('chr1', '737234', '737714'): [('chr1', '736234', '737234'),
  ('chr1', '737714', '738714')],
 ('chr1', '5129446', '5129882'): [('chr1', '5128446', '5129446'),
  ('chr1', '5129882', '5130882')],
 ('chr1', '5321743', '5324308'): [('chr1', '5320743', '5321743'),
  ('chr1', '5324308', '5325308')],
 ('chr1', '5723596', '5724821'): [('chr1', '5722596', '5723596'),
  ('chr1', '5724821', '5725821')],
 ('chr1', '6681571', '6682924'): [('chr1', '6680571', '6681571'),
  ('chr1', '6682924', '6683924')],
 ('chr1', '6863076', '6864098'): [('chr1', '6862076', '6863076'),
  ('chr1', '6864098', '6865098')],
 ('chr1', '6892121', '6892855'): [('chr1', '6891121', '6892121'),
  ('chr1', '6892855', '6893855')],
 ('chr1', '7316394', '7317488'): [('chr1', '7315394', '7316394'),
  ('chr1', '7317488', '7318488')],
 ('chr1'

In [38]:

# mm2_flank_liftover.sb 

In [39]:
# fox analysis

In [40]:
map_found = {}
for dog_rc, flanks in mischka_rc_flanks.items():
    map_found[(dog_rc, flanks[0], flanks[1])] = [0,0]
f = open("blat_cat_pb/mischka_to_fox.paf", 'rt')
for line in f:
    line = line.rstrip().split()
    if int(line[11]) == 255:
        continue
    dog_flank = line[0]
    dog_chro = line[0].split(":")[0]
    dog_start = str(int(line[0].split(":")[1].split("-")[0]))
    dog_end = str(int(line[0].split("-")[1]))
    dog_rc_flank = (dog_chro, dog_start, dog_end) 
    #print(dog_rc_flank)
    power = int(line[11])
    
    genome_chro = line[5]
    genome_start = line[7]
    genome_end = line[8]
    genome_flank = (genome_chro, genome_start, genome_end)
    
    for flanks, checker in map_found.items():
        if dog_rc_flank in flanks:
            if dog_rc_flank == flanks[1]:
                checker[0] = genome_flank
            elif dog_rc_flank == flanks[2]:
                checker[1] = genome_flank
f.close()
map_found

{(('chr1', '325443', '330783'),
  ('chr1', '324443', '325443'),
  ('chr1', '330783', '331783')): [('NW_020356786.1', '103125', '103787'),
  ('NW_020356742.1', '59100', '60100')],
 (('chr1', '408403', '409542'),
  ('chr1', '407403', '408403'),
  ('chr1', '409542', '410542')): [0, 0],
 (('chr1', '737234', '737714'),
  ('chr1', '736234', '737234'),
  ('chr1', '737714', '738714')): [('NW_020356460.1', '24517776', '24518777'),
  ('NW_020356460.1', '24516274', '24517285')],
 (('chr1', '5129446', '5129882'),
  ('chr1', '5128446', '5129446'),
  ('chr1', '5129882', '5130882')): [('NW_020356460.1', '20121093', '20121631'),
  ('NW_020356460.1', '20119357', '20120360')],
 (('chr1', '5321743', '5324308'),
  ('chr1', '5320743', '5321743'),
  ('chr1', '5324308', '5325308')): [('NW_020356460.1', '19933740', '19934726'),
  ('NW_020356460.1', '19932753', '19933755')],
 (('chr1', '5723596', '5724821'),
  ('chr1', '5722596', '5723596'),
  ('chr1', '5724821', '5725821')): [('NW_020356460.1', '19539259', '1

In [41]:
missing = 0
total_found = 0
for dog_coord, genome_coord in map_found.items():
    if 0 in genome_coord:
        missing += 1
        continue
    else:
        total_found += 1
print("total:", total_found, missing, total_found + missing)

total: 3551 107 3658


In [42]:
# check for validity

In [43]:
# pull the kept sequences, check size of intervening sequence
dog_to_fox_valid_rcs = {}
for dog_rc, fox_rc in map_found.items():
    if 0 in fox_rc:
        continue
    
    dog_rc_size = int(dog_rc[0][2]) - int(dog_rc[0][1])
    min_size = round(dog_rc_size * 0.75)
    
    fox_chro_a = fox_rc[0][0]
    fox_chro_b = fox_rc[1][0]
    if fox_chro_a != fox_chro_b: #no matching chro
        continue
    
    fox_aflank_start = int(fox_rc[0][1])
    fox_aflank_end = int(fox_rc[0][2])
    fox_bflank_start = int(fox_rc[1][1])
    fox_bflank_end = int(fox_rc[1][2])
    
    if fox_aflank_end < fox_bflank_start:
        fox_size = fox_bflank_start - fox_aflank_end
        if fox_size < min_size or fox_size > 10000:
            continue
    elif fox_bflank_end < fox_aflank_start:
        fox_size = fox_aflank_start - fox_bflank_end
        if fox_size < min_size or fox_size > 10000:
            continue
            
    dog_to_fox_valid_rcs[dog_rc] = fox_rc
dog_to_fox_valid_rcs

{(('chr1', '737234', '737714'),
  ('chr1', '736234', '737234'),
  ('chr1', '737714', '738714')): [('NW_020356460.1', '24517776', '24518777'),
  ('NW_020356460.1', '24516274', '24517285')],
 (('chr1', '5129446', '5129882'),
  ('chr1', '5128446', '5129446'),
  ('chr1', '5129882', '5130882')): [('NW_020356460.1', '20121093', '20121631'),
  ('NW_020356460.1', '20119357', '20120360')],
 (('chr1', '5723596', '5724821'),
  ('chr1', '5722596', '5723596'),
  ('chr1', '5724821', '5725821')): [('NW_020356460.1', '19539259', '19540196'),
  ('NW_020356460.1', '19537032', '19538037')],
 (('chr1', '6681571', '6682924'),
  ('chr1', '6680571', '6681571'),
  ('chr1', '6682924', '6683924')): [('NW_020356460.1', '18611371', '18612371'),
  ('NW_020356460.1', '18609107', '18610103')],
 (('chr1', '6863076', '6864098'),
  ('chr1', '6862076', '6863076'),
  ('chr1', '6864098', '6865098')): [('NW_020356460.1', '18422574', '18423584'),
  ('NW_020356460.1', '18420552', '18421550')],
 (('chr1', '7316394', '7317488'

In [44]:
len(dog_to_fox_valid_rcs)

1169

In [45]:
# dhole

In [46]:
map_found = {}
for dog_rc, flanks in mischka_rc_flanks.items():
    map_found[(dog_rc, flanks[0], flanks[1])] = [0,0]
f = open("blat_cat_pb/mischka_to_dhole.paf", 'rt')
for line in f:
    line = line.rstrip().split()
    if int(line[11]) == 255:
        continue
    dog_flank = line[0]
    dog_chro = line[0].split(":")[0]
    dog_start = str(int(line[0].split(":")[1].split("-")[0]))
    dog_end = str(int(line[0].split("-")[1]))
    dog_rc_flank = (dog_chro, dog_start, dog_end) 
    #print(dog_rc_flank)
    power = int(line[11])
    
    genome_chro = line[5]
    genome_start = line[7]
    genome_end = line[8]
    genome_flank = (genome_chro, genome_start, genome_end)
    
    for flanks, checker in map_found.items():
        if dog_rc_flank in flanks:
            if dog_rc_flank == flanks[1]:
                checker[0] = genome_flank
            elif dog_rc_flank == flanks[2]:
                checker[1] = genome_flank
f.close()
map_found

{(('chr1', '325443', '330783'),
  ('chr1', '324443', '325443'),
  ('chr1', '330783', '331783')): [0, ('GWHAAAC00000995', '37276', '38265')],
 (('chr1', '408403', '409542'),
  ('chr1', '407403', '408403'),
  ('chr1', '409542', '410542')): [('GWHAAAC00000995', '109972', '110972'),
  ('GWHAAAC00000995', '109649', '109974')],
 (('chr1', '737234', '737714'),
  ('chr1', '736234', '737234'),
  ('chr1', '737714', '738714')): [('GWHAAAC00000226', '112723', '113724'),
  ('GWHAAAC00000226', '114197', '115209')],
 (('chr1', '5129446', '5129882'),
  ('chr1', '5128446', '5129446'),
  ('chr1', '5129882', '5130882')): [('GWHAAAC00000226', '4472379', '4473380'),
  ('GWHAAAC00000226', '4473800', '4474801')],
 (('chr1', '5321743', '5324308'),
  ('chr1', '5320743', '5321743'),
  ('chr1', '5324308', '5325308')): [('GWHAAAC00000226', '4656921', '4657914'),
  ('GWHAAAC00000226', '4657899', '4658888')],
 (('chr1', '5723596', '5724821'),
  ('chr1', '5722596', '5723596'),
  ('chr1', '5724821', '5725821')): [('G

In [47]:
missing = 0
total_found = 0
for dog_coord, genome_coord in map_found.items():
    if 0 in genome_coord:
        missing += 1
        continue
    else:
        total_found += 1
print("total:", total_found, missing, total_found + missing)

total: 3588 70 3658


In [48]:
# pull the kept sequences, check size of intervening sequence
dog_to_dhole_valid_rcs = {}
for dog_rc, dhole_rc in map_found.items():
    if 0 in dhole_rc:
        continue
    
    dog_rc_size = int(dog_rc[0][2]) - int(dog_rc[0][1])
    min_size = round(dog_rc_size * 0.75)
    
    dhole_chro_a = dhole_rc[0][0]
    dhole_chro_b = dhole_rc[1][0]
    if dhole_chro_a != dhole_chro_b: #no matching chro
        continue
    
    dhole_aflank_start = int(dhole_rc[0][1])
    dhole_aflank_end = int(dhole_rc[0][2])
    dhole_bflank_start = int(dhole_rc[1][1])
    dhole_bflank_end = int(dhole_rc[1][2])
    
    if dhole_aflank_end < dhole_bflank_start:
        dhole_size = dhole_bflank_start - dhole_aflank_end
        if dhole_size < min_size or dhole_size > 10000:
            continue
    elif dhole_bflank_end < dhole_aflank_start:
        dhole_size = dhole_aflank_start - dhole_bflank_end
        if dhole_size < min_size or dhole_size > 10000:
            continue
            
        dog_to_dhole_valid_rcs[dog_rc] = dhole_rc
dog_to_dhole_valid_rcs

{(('chr1', '6681571', '6682924'),
  ('chr1', '6680571', '6681571'),
  ('chr1', '6682924', '6683924')): [('GWHAAAC00000142', '2985512', '2986513'),
  ('GWHAAAC00000142', '2983164', '2984165')],
 (('chr1', '6863076', '6864098'),
  ('chr1', '6862076', '6863076'),
  ('chr1', '6864098', '6865098')): [('GWHAAAC00000142', '2787526', '2788535'),
  ('GWHAAAC00000142', '2785564', '2786565')],
 (('chr1', '6892121', '6892855'),
  ('chr1', '6891121', '6892121'),
  ('chr1', '6892855', '6893855')): [('GWHAAAC00000142', '2755155', '2756175'),
  ('GWHAAAC00000142', '2753413', '2754413')],
 (('chr1', '7316394', '7317488'),
  ('chr1', '7315394', '7316394'),
  ('chr1', '7317488', '7318488')): [('GWHAAAC00000142', '2337795', '2338648'),
  ('GWHAAAC00000142', '2335776', '2336771')],
 (('chr1', '8098375', '8098750'),
  ('chr1', '8097375', '8098375'),
  ('chr1', '8098750', '8099750')): [('GWHAAAC00000142', '1559565', '1560562'),
  ('GWHAAAC00000142', '1558327', '1559222')],
 (('chr1', '9335741', '9337412'),
 

In [49]:
len(dog_to_dhole_valid_rcs)

1444

In [50]:
dog_to_fox_valid_rcs

{(('chr1', '737234', '737714'),
  ('chr1', '736234', '737234'),
  ('chr1', '737714', '738714')): [('NW_020356460.1', '24517776', '24518777'),
  ('NW_020356460.1', '24516274', '24517285')],
 (('chr1', '5129446', '5129882'),
  ('chr1', '5128446', '5129446'),
  ('chr1', '5129882', '5130882')): [('NW_020356460.1', '20121093', '20121631'),
  ('NW_020356460.1', '20119357', '20120360')],
 (('chr1', '5723596', '5724821'),
  ('chr1', '5722596', '5723596'),
  ('chr1', '5724821', '5725821')): [('NW_020356460.1', '19539259', '19540196'),
  ('NW_020356460.1', '19537032', '19538037')],
 (('chr1', '6681571', '6682924'),
  ('chr1', '6680571', '6681571'),
  ('chr1', '6682924', '6683924')): [('NW_020356460.1', '18611371', '18612371'),
  ('NW_020356460.1', '18609107', '18610103')],
 (('chr1', '6863076', '6864098'),
  ('chr1', '6862076', '6863076'),
  ('chr1', '6864098', '6865098')): [('NW_020356460.1', '18422574', '18423584'),
  ('NW_020356460.1', '18420552', '18421550')],
 (('chr1', '7316394', '7317488'

In [51]:
fox_to_dog_rcs = {}
f = open("blat_cat_pb/fox_to_mischka.paf", 'rt')
for line in f:
    line = line.rstrip().split()
    fox_chro = line[0].split(":")[0]
    fox_start = line[0].split(":")[1].split("-")[0]
    fox_end = line[0].split("-")[1]
    fox_rc = (fox_chro, fox_start, fox_end)
    dog_rc = (line[5], line[7], line[8])

    quality = int(line[11])
    size = int(line[8]) - int(line[7])        
    if quality == 255:
        continue

    if fox_rc not in fox_to_dog_rcs.keys():
        fox_to_dog_rcs[fox_rc] = [dog_rc, quality, size]
    else:
        if quality > fox_to_dog_rcs[fox_rc][1]:
            fox_to_dog_rcs[fox_rc] = [dog_rc, quality, size]
        elif quality == fox_to_dog_rcs[fox_rc][1] and size > fox_to_dog_rcs[fox_rc][2]:
            fox_to_dog_rcs[fox_rc] = [dog_rc, quality, size]
f.close()
fox_to_dog_rcs

{('NW_020356460.1', '24517285', '24517776'): [('chr1', '737234', '737714'),
  60,
  480],
 ('NW_020356460.1', '20120360', '20121093'): [('chr1', '5129180', '5129883'),
  60,
  703],
 ('NW_020356460.1', '19538037', '19539259'): [('chr1', '5723609', '5724821'),
  60,
  1212],
 ('NW_020356460.1', '18610103', '18611371'): [('chr1', '6681571', '6682926'),
  60,
  1355],
 ('NW_020356460.1', '18421550', '18422574'): [('chr1', '6863076', '6864095'),
  60,
  1019],
 ('NW_020356460.1', '17981270', '17982718'): [('chr1', '7316229', '7317488'),
  60,
  1259],
 ('NW_020356460.1', '17181513', '17181941'): [('chr1', '8098345', '8098768'),
  22,
  423],
 ('NW_020356460.1', '15951095', '15952782'): [('chr1', '9335732', '9337412'),
  1,
  1680],
 ('NW_020356460.1', '15211473', '15211941'): [('chr1', '10074010', '10074428'),
  60,
  418],
 ('NW_020356460.1', '11084808', '11085496'): [('chr1', '14211117', '14211771'),
  17,
  654],
 ('NW_020356460.1', '9047731', '9048324'): [('chr1', '16250498', '16251091

In [52]:
dog_to_fox_to_dog_rcs = {}
no_match = 0
for dog_rc, fox_rc in dog_to_fox_valid_rcs.items():
    try:
        if int(fox_rc[0][2]) < int(fox_rc[1][1]):
            fox_location = (fox_rc[0][0], fox_rc[0][2], fox_rc[1][1])
        else:
            fox_location = (fox_rc[0][0], fox_rc[1][2], fox_rc[0][1])
        #print(fox_rc)
        dog_lift = fox_to_dog_rcs[fox_location]
        if dog_rc in dog_to_fox_to_dog_rcs.keys():
            print(dog_rc)
        dog_to_fox_to_dog_rcs[dog_rc] = [fox_location, dog_lift]
    except:
        no_match += 1
dog_to_fox_to_dog_rcs

{(('chr1', '737234', '737714'),
  ('chr1', '736234', '737234'),
  ('chr1', '737714', '738714')): [('NW_020356460.1', '24517285', '24517776'),
  [('chr1', '737234', '737714'), 60, 480]],
 (('chr1', '5129446', '5129882'),
  ('chr1', '5128446', '5129446'),
  ('chr1', '5129882', '5130882')): [('NW_020356460.1', '20120360', '20121093'),
  [('chr1', '5129180', '5129883'), 60, 703]],
 (('chr1', '5723596', '5724821'),
  ('chr1', '5722596', '5723596'),
  ('chr1', '5724821', '5725821')): [('NW_020356460.1', '19538037', '19539259'),
  [('chr1', '5723609', '5724821'), 60, 1212]],
 (('chr1', '6681571', '6682924'),
  ('chr1', '6680571', '6681571'),
  ('chr1', '6682924', '6683924')): [('NW_020356460.1', '18610103', '18611371'),
  [('chr1', '6681571', '6682926'), 60, 1355]],
 (('chr1', '6863076', '6864098'),
  ('chr1', '6862076', '6863076'),
  ('chr1', '6864098', '6865098')): [('NW_020356460.1', '18421550', '18422574'),
  [('chr1', '6863076', '6864095'), 60, 1019]],
 (('chr1', '7316394', '7317488'),
 

In [53]:
no_match

32

In [54]:
len(dog_to_fox_to_dog_rcs)

1137

In [55]:
def is_overlapping(x1,x2,y1,y2):
    checker = (max(x1,y1) <= min(x2,y2))
    if checker is True:
        return min(x2,y2) - max(x1,y1)
    else:
        return False

In [56]:
# now check to see if orig rc and mapped rc are the same location
no_match = 0
fox_overlap_found = {}
for dog_rc, lifted_rcs in dog_to_fox_to_dog_rcs.items():
    fox_rc = lifted_rcs[0]
    lift_dog = lifted_rcs[1]

    orig_chro = dog_rc[0][0]
    new_chro = lift_dog[0][0]

    if orig_chro != new_chro: # fox rc lifted to a different chro than original
        no_match += 1
        continue
        
    orig_start = int(dog_rc[0][1])
    new_start = int(lift_dog[0][1])
    orig_end = int(dog_rc[0][2]) 
    new_end = int(lift_dog[0][2])

    #print(orig_chro, orig_start, orig_end, new_start, new_end)

    if is_overlapping(orig_start, orig_end, new_start, new_end) is False: #mapped rc must overlap original
        no_match += 1
        continue
    else:
        fox_overlap_found[dog_rc] = [fox_rc, lift_dog]
no_match

120

In [57]:
len(fox_overlap_found) #1017 of the original 3658 mischka retrocopies have a fox counterpart

1017

In [58]:
dhole_to_dog_rcs = {}
f = open("blat_cat_pb/dhole_to_mischka.paf", 'rt')
for line in f:
    line = line.rstrip().split()
    dhole_chro = line[0].split(":")[0]
    dhole_start = line[0].split(":")[1].split("-")[0]
    dhole_end = line[0].split("-")[1]
    dhole_rc = (dhole_chro, dhole_start, dhole_end)
    dog_rc = (line[5], line[7], line[8])

    quality = int(line[11])
    size = int(line[8]) - int(line[7])        
    if quality == 255:
        continue

    if dhole_rc not in dhole_to_dog_rcs.keys():
        dhole_to_dog_rcs[dhole_rc] = [dog_rc, quality, size]
    else:
        if quality > dhole_to_dog_rcs[dhole_rc][1]:
            dhole_to_dog_rcs[dhole_rc] = [dog_rc, quality, size]
        elif quality == dhole_to_dog_rcs[dhole_rc][1] and size > dhole_to_dog_rcs[dhole_rc][2]:
            dhole_to_dog_rcs[dhole_rc] = [dog_rc, quality, size]
f.close()
dhole_to_dog_rcs

{('GWHAAAC00000142', '2984165', '2985512'): [('chr1', '6681571', '6682924'),
  60,
  1353],
 ('GWHAAAC00000142', '2786565', '2787526'): [('chr1', '6863076', '6864098'),
  60,
  1022],
 ('GWHAAAC00000142', '2754413', '2755155'): [('chr1', '6892110', '6892855'),
  60,
  745],
 ('GWHAAAC00000142', '2336771', '2337795'): [('chr1', '7316470', '7317488'),
  60,
  1018],
 ('GWHAAAC00000142', '1559222', '1559565'): [('chr1', '8098375', '8098717'),
  60,
  342],
 ('GWHAAAC00000142', '348119', '349786'): [('chr1', '9335741', '9337412'),
  60,
  1671],
 ('GWHAAAC00000615', '956408', '956916'): [('chr1', '10073974', '10074481'),
  60,
  507],
 ('GWHAAAC00000652', '430157', '430585'): [('chrX', '59657692', '59658121'),
  26,
  429],
 ('GWHAAAC00000823', '420771', '421260'): [('chr1', '25813048', '25813537'),
  11,
  489],
 ('GWHAAAC00000277', '1609801', '1610733'): [('chr1', '27021055', '27022068'),
  60,
  1013],
 ('GWHAAAC00000277', '1525817', '1526470'): [('chr1', '27106129', '27106782'),
  60,


In [59]:
dog_to_dhole_valid_rcs

{(('chr1', '6681571', '6682924'),
  ('chr1', '6680571', '6681571'),
  ('chr1', '6682924', '6683924')): [('GWHAAAC00000142', '2985512', '2986513'),
  ('GWHAAAC00000142', '2983164', '2984165')],
 (('chr1', '6863076', '6864098'),
  ('chr1', '6862076', '6863076'),
  ('chr1', '6864098', '6865098')): [('GWHAAAC00000142', '2787526', '2788535'),
  ('GWHAAAC00000142', '2785564', '2786565')],
 (('chr1', '6892121', '6892855'),
  ('chr1', '6891121', '6892121'),
  ('chr1', '6892855', '6893855')): [('GWHAAAC00000142', '2755155', '2756175'),
  ('GWHAAAC00000142', '2753413', '2754413')],
 (('chr1', '7316394', '7317488'),
  ('chr1', '7315394', '7316394'),
  ('chr1', '7317488', '7318488')): [('GWHAAAC00000142', '2337795', '2338648'),
  ('GWHAAAC00000142', '2335776', '2336771')],
 (('chr1', '8098375', '8098750'),
  ('chr1', '8097375', '8098375'),
  ('chr1', '8098750', '8099750')): [('GWHAAAC00000142', '1559565', '1560562'),
  ('GWHAAAC00000142', '1558327', '1559222')],
 (('chr1', '9335741', '9337412'),
 

In [60]:
dog_to_dhole_to_dog_rcs = {}
no_match = 0
for dog_rc, dhole_rc in dog_to_dhole_valid_rcs.items():
    try:
        if int(dhole_rc[0][2]) < int(dhole_rc[1][1]):
            dhole_location = (dhole_rc[0][0], dhole_rc[0][2], dhole_rc[1][1])
        else:
            dhole_location = (dhole_rc[0][0], dhole_rc[1][2], dhole_rc[0][1])
        #print(dhole_rc)
        dog_lift = dhole_to_dog_rcs[dhole_location]
        if dog_rc in dog_to_dhole_to_dog_rcs.keys():
            print(dog_rc)
        dog_to_dhole_to_dog_rcs[dog_rc] = [dhole_location, dog_lift]
    except:
        no_match += 1
dog_to_dhole_to_dog_rcs

{(('chr1', '6681571', '6682924'),
  ('chr1', '6680571', '6681571'),
  ('chr1', '6682924', '6683924')): [('GWHAAAC00000142', '2984165', '2985512'),
  [('chr1', '6681571', '6682924'), 60, 1353]],
 (('chr1', '6863076', '6864098'),
  ('chr1', '6862076', '6863076'),
  ('chr1', '6864098', '6865098')): [('GWHAAAC00000142', '2786565', '2787526'),
  [('chr1', '6863076', '6864098'), 60, 1022]],
 (('chr1', '6892121', '6892855'),
  ('chr1', '6891121', '6892121'),
  ('chr1', '6892855', '6893855')): [('GWHAAAC00000142', '2754413', '2755155'),
  [('chr1', '6892110', '6892855'), 60, 745]],
 (('chr1', '7316394', '7317488'),
  ('chr1', '7315394', '7316394'),
  ('chr1', '7317488', '7318488')): [('GWHAAAC00000142', '2336771', '2337795'),
  [('chr1', '7316470', '7317488'), 60, 1018]],
 (('chr1', '8098375', '8098750'),
  ('chr1', '8097375', '8098375'),
  ('chr1', '8098750', '8099750')): [('GWHAAAC00000142', '1559222', '1559565'),
  [('chr1', '8098375', '8098717'), 60, 342]],
 (('chr1', '9335741', '9337412')

In [61]:
no_match

23

In [62]:
len(dog_to_dhole_to_dog_rcs)

1421

In [63]:
# now check to see if orig rc and mapped rc are the same location
no_match = 0
dhole_overlap_found = {}
for dog_rc, lifted_rcs in dog_to_dhole_to_dog_rcs.items():
    dhole_rc = lifted_rcs[0]
    lift_dog = lifted_rcs[1]

    orig_chro = dog_rc[0][0]
    new_chro = lift_dog[0][0]

    if orig_chro != new_chro: # dhole rc lifted to a different chro than original
        no_match += 1
        continue
        
    orig_start = int(dog_rc[0][1])
    new_start = int(lift_dog[0][1])
    orig_end = int(dog_rc[0][2]) 
    new_end = int(lift_dog[0][2])

    #print(orig_chro, orig_start, orig_end, new_start, new_end)

    if is_overlapping(orig_start, orig_end, new_start, new_end) is False: #mapped rc must overlap original
        no_match += 1
        continue
    else:
        dhole_overlap_found[dog_rc] = [dhole_rc, lift_dog]
no_match

47

In [64]:
len(dhole_overlap_found) #1374 of the original 3658 mischka retrocopies have a fox counterpart

1374

In [65]:
# find unique
fox_union = 0
dhole_union = 0
fox_unique = 0
dhole_unique = 0
union_rcs = {}
fox_unique_rcs = {}
dhole_unique_rcs = {}
for dog_rc in fox_overlap_found.keys():
    if dog_rc in dhole_overlap_found.keys():
        fox_union += 1
        union_rcs[dog_rc[0]] = [fox_overlap_found[dog_rc][0], dhole_overlap_found[dog_rc][0]]
    else:
        fox_unique += 1
        fox_unique_rcs[dog_rc[0]] = fox_overlap_found[dog_rc][0]
for dog_rc in dhole_overlap_found.keys():
    if dog_rc in fox_overlap_found.keys():
        dhole_union += 1
    else:
        dhole_unique += 1
        dhole_unique_rcs[dog_rc[0]] = dhole_overlap_found[dog_rc][0]
fox_union, dhole_union, fox_unique, dhole_unique, fox_union + fox_unique + dhole_unique

(468, 468, 549, 906, 1923)

In [66]:
len(union_rcs) / 3658

0.12793876435210497

In [67]:
len(fox_unique_rcs)/ 3658

0.15008201202843083

In [68]:
len(dhole_unique_rcs)/ 3658

0.2476763258611263

In [69]:
# 1923 of 3658 (52.57%) of Mischka retrocopies have a dhole and/or fox counterpart
# 468 shared between fox and dhole
# 549 unique to fox
# 906 unique to dhole 

In [70]:
union_rcs

{('chr1', '6681571', '6682924'): [('NW_020356460.1', '18610103', '18611371'),
  ('GWHAAAC00000142', '2984165', '2985512')],
 ('chr1', '6863076', '6864098'): [('NW_020356460.1', '18421550', '18422574'),
  ('GWHAAAC00000142', '2786565', '2787526')],
 ('chr1', '7316394', '7317488'): [('NW_020356460.1', '17981270', '17982718'),
  ('GWHAAAC00000142', '2336771', '2337795')],
 ('chr1', '8098375', '8098750'): [('NW_020356460.1', '17181513', '17181941'),
  ('GWHAAAC00000142', '1559222', '1559565')],
 ('chr1', '9335741', '9337412'): [('NW_020356460.1', '15951095', '15952782'),
  ('GWHAAAC00000142', '348119', '349786')],
 ('chr1', '10073974', '10074431'): [('NW_020356460.1', '15211473', '15211941'),
  ('GWHAAAC00000615', '956408', '956916')],
 ('chr1', '47792179', '47795237'): [('NW_020356592.1', '2138374', '2141435'),
  ('GWHAAAC00000188', '3595199', '3597630')],
 ('chr1', '48181739', '48183315'): [('NW_020356592.1', '1744483', '1746319'),
  ('GWHAAAC00000188', '3203070', '3204645')],
 ('chr1', 

In [71]:
fox_unique_rcs

{('chr1', '737234', '737714'): ('NW_020356460.1', '24517285', '24517776'),
 ('chr1', '5129446', '5129882'): ('NW_020356460.1', '20120360', '20121093'),
 ('chr1', '5723596', '5724821'): ('NW_020356460.1', '19538037', '19539259'),
 ('chr1', '14211120', '14211771'): ('NW_020356460.1', '11084808', '11085496'),
 ('chr1', '16250498', '16251091'): ('NW_020356460.1', '9047731', '9048324'),
 ('chr1', '16393477', '16394886'): ('NW_020356460.1', '8885419', '8886811'),
 ('chr1', '19233451', '19234324'): ('NW_020356460.1', '5980152', '5981177'),
 ('chr1', '60887629', '60887933'): ('NW_020356475.1', '11065628', '11066073'),
 ('chr1', '78973258', '78974397'): ('NW_020356690.1', '301857', '303007'),
 ('chr1', '79040595', '79042372'): ('NW_020356690.1', '228576', '230345'),
 ('chr1', '79045299', '79046686'): ('NW_020356690.1', '224172', '225656'),
 ('chr1', '96037340', '96037962'): ('NW_020356627.1', '2450262', '2451059'),
 ('chr1', '96132002', '96132391'): ('NW_020356627.1', '2355412', '2355779'),
 ('

In [74]:
# find overlap with mischka fullORFs
f = open("mischka.fullORF_retrocopies.txt", 'rt')
msk_fullORF = []
for line in f:
    line = line.rstrip().split()
    msk_fullORF.append(line)
    #print(line)
f.close()
msk_fullORF

[['chr24', '32101391', '32102484', 'GAPDH'],
 ['chr19', '36773904', '36774333', 'RPL31'],
 ['chrX', '59657693', '59658122', 'RPL31'],
 ['chr12', '11811454', '11812310', 'OSTC'],
 ['chr5', '55519969', '55520676', 'PFDN1'],
 ['chr24', '24640181', '24640730', 'RPL21'],
 ['chr26', '14097621', '14098169', 'RPL21'],
 ['chr37', '9640899', '9642140', 'RPL21'],
 ['chr11', '61894770', '61896533', 'EEF1A1'],
 ['chr15', '7550735', '7552827', 'EEF1A1'],
 ['chr4', '68625846', '68627608', 'EEF1A1'],
 ['chrX', '10660657', '10661428', 'GSTP1'],
 ['chr8', '8222137', '8223548', 'PRR3'],
 ['chr1', '39885929', '39886834', 'APOO'],
 ['chr18', '13507467', '13508372', 'APOO'],
 ['chr9', '53897469', '53898412', 'SNRPB2'],
 ['chr28', '8377290', '8378487', 'HNRNPA3'],
 ['chr7', '38845030', '38846100', 'NSA2'],
 ['chr20', '5410644', '5411154', 'RPL26'],
 ['chr24', '16359571', '16360079', 'RPL26'],
 ['chr11', '12175194', '12175911', 'RPL10A'],
 ['chr12', '39236779', '39237126', 'RPL37'],
 ['chr16', '13985503', '13

In [78]:
union_ORF = []
fox_ORF = []
dhole_ORF = []
no_lift = 0
for rc in msk_fullORF:
    if rc[0:3] in list(union_rcs.keys()):
        union_ORF.append(rc)
    elif rc[0:3] in list(fox_unique_rcs.keys()):
        fox_ORF.append(rc)
    elif rc[0:3] in list(dhole_unique_rcs.keys()):
        dhole_ORF.append(rc)
    else:
        no_lift += 1
len(union_ORF), len(fox_ORF), len(dhole_ORF), no_lift

(0, 0, 0, 66)